# Creating AML Environment
Before we get started in building out the training script, we want to create an AML environment. This will ensure we have a consistent training environment to help get repeatable code.

In [2]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()

#### 1. Create Static Variables
* Read DockerFile to string
* Set CONDA_PACKAGES
* Set PIP_PACKAGES
* Set PYTHON_VERSION

In [3]:
ENVIRONMENT_NAME = 'lightgbm-cli'

CONDA_PACKAGES = []
PIP_PACKAGES = ['click==7.1.1', 'azureml-defaults==1.2.0']
PYTHON_VERSION = '3.7.7'

In [4]:
BASE_DOCKER_IMAGE = 'smartassistaa760ea9a.azurecr.io/lightgbm:latest'

In [5]:
# Read in the text of the Dockerfile
with open('../Dockerfile', 'r') as fo:
    DOCKERFILE = fo.read()

print(DOCKERFILE)

# Modified from Dockerfile found at https://github.com/microsoft/LightGBM/blob/master/docker/dockerfile-cli (commit 83ecb38)
FROM mcr.microsoft.com/azureml/base:openmpi3.1.2-ubuntu16.04

RUN apt-get update && \
    apt-get install -y --no-install-recommends \
        ca-certificates \
        cmake \
        build-essential \
        gcc \
        g++ \
        git && \
    rm -rf /var/lib/apt/lists/*

RUN git clone --recursive --branch stable --depth 1 https://github.com/Microsoft/LightGBM && \
    mkdir LightGBM/build && \
    cd LightGBM/build && \
    cmake .. && \
    make -j $(nproc) && \
    make install && \
    cd ../.. && \
    rm -rf LightGBM


#### 2. Create an AML Environment object and overwrite the subsections

In [6]:
lgbm_env = Environment(ENVIRONMENT_NAME)

##### 2.1 Python Settings

In [7]:
from azureml.core.environment import CondaDependencies

lgbm_env.python.conda_dependencies = CondaDependencies.create(python_version=PYTHON_VERSION, 
                                                              conda_packages=CONDA_PACKAGES, 
                                                              pip_packages=PIP_PACKAGES)

##### 2.2 Docker Settings

In [8]:
# Set Docker to be enabled by default
lgbm_env.docker.enabled = True

# baseImage and baseDockerfile are mutually exclusive
lgbm_env.docker.base_image = BASE_DOCKER_IMAGE
lgbm_env.docker.base_dockerfile = None

In [9]:
lgbm_env

{
    "name": "lightgbm-cli",
    "version": null,
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "name": "project_environment",
            "dependencies": [
                "python=3.7.7",
                {
                    "pip": [
                        "click==7.1.1",
                        "azureml-defaults==1.2.0"
                    ]
                }
            ],
            "channels": [
                "anaconda",
                "conda-forge"
            ]
        }
    },
    "docker": {
        "enabled": true,
        "baseImage": "smartassistaa760ea9a.azurecr.io/lightgbm:latest",
        "baseDockerfile": null,
        "sharedVolumes": true,
        "shmSize": "2g",
        "arguments": [],
        "baseImageRegistry":

#### 3. Register the environment to the workspace

In [10]:
lgbm_env = lgbm_env.register(ws)

#### 4. Build the container
This isn't strictly required at this point. If you don't explicitly build the container now, it will be built upon first invocation. I tend to explicitly build the containers to (a) speed up the first training submission and (b) to seperate failures from my environment from failures in my code.

The `build()` command _doesn't require Docker to be installed locally_, instead, this build is being done by the Azure Container Registry.

In [11]:
build_status = lgbm_env.build(ws)
build_status.wait_for_completion(show_output=True)

Image Build Status: Running

2020/04/16 18:42:21 Downloading source code...
2020/04/16 18:42:22 Finished downloading source code
2020/04/16 18:42:23 Creating Docker network: acb_default_network, driver: 'bridge'
2020/04/16 18:42:23 Successfully set up Docker network: acb_default_network
2020/04/16 18:42:23 Setting up Docker configuration...
2020/04/16 18:42:24 Successfully set up Docker configuration
2020/04/16 18:42:24 Logging in to registry: smartassistaa760ea9a.azurecr.io
2020/04/16 18:42:25 Successfully logged into smartassistaa760ea9a.azurecr.io
2020/04/16 18:42:25 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/04/16 18:42:25 Launching container with name: acb_step_0
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /acb/home/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/

#### 5. Check the container is built and saved in the Azure Container Registry
If you see `'imageExistsInRegistry': True` in the `get_image_details()` call, then your container is stored in the registry and should be pulled from there upon job submission

In [12]:
import pprint
pp = pprint.PrettyPrinter(depth=5)
pp.pprint(lgbm_env.get_image_details(ws))

{'dockerImage': {'name': 'azureml/azureml_530807073dcffa9aa4f21724cf8481da',
                 'registry': {'address': 'smartassistaa760ea9a.azurecr.io',
                              'password': 'ELwuS0NsA8U=aE+owbkkSwo0cqfvf6sK',
                              'username': 'smartassistaa760ea9a'}},
 'imageExistsInRegistry': True,
 'ingredients': {'condaSpecification': 'channels:\n'
                                       '- anaconda\n'
                                       '- conda-forge\n'
                                       'dependencies:\n'
                                       '- python=3.7.7\n'
                                       '- pip:\n'
                                       '  - click==7.1.1\n'
                                       '  - azureml-defaults==1.2.0\n'
                                       'name: '
                                       'azureml_f93775a230cd65f7f04349c78d61d976\n',
                 'dockerfile': 'FROM '
                               'smart

<br><br><br><br><br><br>
###### Copyright (c) 2020 Microsoft

###### This software is released under the MIT License.
###### https://opensource.org/licenses/MIT